In [ ]:
# This is comment
#second line comment
import os
import cv2
import numpy as np
from ultralytics import YOLO

# --- USER: VERIFY YOUR PROJECT DIRECTORY ---
PROJECT_DIR = "/content/drive/MyDrive/Colab_Projects/AI-Based-Intelligent-Traffic-Monitoring-and-Management-System"
# The name of your input video file, which should be in the 'data' subfolder
INPUT_VIDEO_FILENAME = "input.mp4"
# ---

# --- USER: DEFINE YOUR LANE'S ROI HERE ---
# Define the coordinates of YOUR lane's polygon
ROI_POINTS = np.array([(100, 300), (800, 300), (850, 450), (50, 450)], np.int32)
# ---

# Construct full paths for video files
input_video_path = os.path.join(PROJECT_DIR, 'data', INPUT_VIDEO_FILENAME)
output_video_path = os.path.join(PROJECT_DIR, 'output', 'output_vehicle_count.mp4')

# Ensure the output directory exists
os.makedirs(os.path.join(PROJECT_DIR, 'output'), exist_ok=True)

# Change the current working directory to the project folder
os.chdir(PROJECT_DIR)

# Load the pre-trained YOLOv8 model
print("Loading YOLOv8 model...")
model = YOLO('yolov8n.pt')

# COCO class IDs for vehicles (car, motorcycle, bus, truck)
VEHICLE_CLASS_IDS = [2, 3, 5, 7]

# Video Processing

In [ ]:
# Open the input video file
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print(f"Error: Could not open video at {input_video_path}")
else:
    # Get video properties for the output file
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"Processing video: {INPUT_VIDEO_FILENAME}...")

try:
    while True:
        success, frame = cap.read()
        if not success:
            break # End of video

        # This check prevents errors if a frame is corrupted
        if frame.shape[1] != frame_width or frame.shape[0] != frame_height:
            frame = cv2.resize(frame, (frame_width, frame_height))

        # Reset counter for each frame
        vehicle_count = 0

        # Draw the ROI polygon once per frame
        cv2.polylines(frame, [ROI_POINTS], isClosed=True, color=(255, 0, 0), thickness=2)

        # Run YOLO detection
        results = model(frame)

        # Process detections
        for box in results[0].boxes:
            if int(box.cls) in VEHICLE_CLASS_IDS:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

                if cv2.pointPolygonTest(ROI_POINTS, (center_x, center_y), False) >= 0:
                    vehicle_count += 1
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Write the final count and the frame to the output video
        cv2.putText(frame, f"Vehicles in Lane: {vehicle_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        out.write(frame)

finally:
    # This ensures files are closed properly even if an error occurs
    print("Processing complete. Releasing resources.")
    cap.release()
    out.release()